In [111]:
import tweepy
import pandas as pd
import preprocessor as p
from watson_developer_cloud import PersonalityInsightsV3
from langdetect import detect_langs
from langid.langid import LanguageIdentifier, model
import pandas as pd
import time

### Reading data from the html table

In [112]:
actors = pd.read_html("https://www.socialbakers.com/statistics/twitter/profiles/india/celebrities/actor/page-1-2/")[0]
sports_star = pd.read_html("https://www.socialbakers.com/statistics/twitter/profiles/india/celebrities/sport-star/page-1-2/")[0]
writer = pd.read_html("https://www.socialbakers.com/statistics/twitter/profiles/india/celebrities/writer/page-1-2/")[0]
politicians = pd.read_html("https://www.socialbakers.com/statistics/twitter/profiles/india/society/politics/page-1-2/")[0]

In [1]:
import pandas as pd

In [12]:
actors = pd.read_html("https://www.socialbakers.com/statistics/twitter/profiles/india/celebrities/actor/page-1-2/")[0]

In [18]:
actors.dropna(axis=0)

,0,1,2,3
0,1,1 Amitabh Bachchan (@SrBachchan),Followings Followings 1 164,Followers Followers 31 406 900
1,2,2 Shah Rukh Khan (@iamsrk),Followings 81,Followers 30 750 527
2,3,3 Salman Khan (@BeingSalmanKhan),Followings 22,Followers 28 310 483
3,4,4 Akshay Kumar (@akshaykumar),Followings 25,Followers 22 645 866
4,5,5 Aamir Khan (@aamir_khan),Followings 9,Followers 22 390 346
5,6,6 Deepika Padukone (@deepikapadukone),Followings 74,Followers 21 986 651
6,7,7 Hrithik Roshan (@iHrithik),Followings 86,Followers 20 742 693
7,8,8 PRIYANKA (@priyankachopra),Followings 537,Followers 20 547 906
8,9,9 Alia Bhatt (@aliaa08),Followings 237,Followers 14 690 943
9,10,10 Anushka Sharma (@AnushkaSharma),Followings 133,Followers 13 581 864


#### Renaming the columns in above data frames

In [113]:
data = [actors, sports_star, writer, politicians]
for item in data:
    item.columns = ['rank', 'celebrity', 'followings', 'followers']

# Merging all the categories of celebrities to one
df = pd.concat(data)
celebrities = df.celebrity.dropna(axis=0, how='all')
celebrities.head()

0    1  Amitabh Bachchan (@SrBachchan)
1          2  Shah Rukh Khan (@iamsrk)
2    3  Salman Khan (@BeingSalmanKhan)
3       4  Akshay Kumar (@akshaykumar)
4          5  Aamir Khan (@aamir_khan)
Name: celebrity, dtype: object

#### cleaning the df columns to obtain Twitter user names

In [115]:
celeb_list  = celebrities.str.extract('@(\S+)\)', expand = True)[0].tolist()

In [116]:
consumer_key = '2QTDFjrpIy71bwCuJXwgC85oD'
consumer_secret = '2sctC17IBpkigXU7TbzQnygSK3Uydit9yw4Pn0Xjhc5N8Jaftd'
access_token = '2891438354-ZrPa7xndtycr4DoNiG2JkdQXZ52WrZEIiIRyG3X'
access_secret = '92HfNTEzY9HNrbrHPcvYQPibFtA1BCtzRwPSiVTUETkhv'

### outputs only the Tweets of a specifc user

#### following function uses LanguageIdentifier package for extracting the relevant tweets that have probability of being written in english greater than 0.9 

### Language Detection using langid package gives better accuracy in detecting English language but execution time increases. 


In [117]:
def LanguageIdentifier_relevant_tweets(screen_name):
    #Twitter only allows access to a users most recent tweets with this method

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    relevant_tweet = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200, lang='en', include_rts=False, exclude_replies = True)
    
    #save most recent tweets
    
    alltweets.extend(new_tweets)
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest, lang='en', include_rts=False, exclude_replies = True)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
    
    for tweet in alltweets:
        # Data Cleaning 2> Extracting alphabetic and space text from tweets 
        twt = "".join(x for x in p.clean(tweet.text) if x.isalpha() or x.isspace())
        identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)
        langProb = identifier.classify(twt)
        if langProb[0] == 'en' and float(langProb[1]) > 0.9:
            relevant_tweet.append(twt)
    return relevant_tweet

#### Extracting relevant tweets Using Lang detect package

### Langdetect package has faster execution time but accuracy of identifying English language is lesser

In [118]:
def detect_langs_relevant_tweets(screen_name):
    #Twitter only allows access to a users most recent tweets with this method

    #authorize twitter, initialize tweepy
    start_time = time.time()
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    api = tweepy.API(auth)

    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    relevant_tweet = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200, lang='en', include_rts=False, exclude_replies = True)
    
    #save most recent tweets
    
    alltweets.extend(new_tweets)
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest, lang='en', include_rts=False, exclude_replies = True)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
    
    for tweet in alltweets:
        # Data Cleaning 2> Extracting alphabetic and space text from tweets 
        twt = "".join(x for x in p.clean(tweet.text) if x.isalpha() or x.isspace())
        if (len(twt) > 0):
            if (str(detect_langs(twt)[0])[:2] =='en') & (float(str(detect_langs(twt)[0])[3:]) > 0.9):
                relevant_tweet.append(twt)
    #print('Execution time for relevant_tweets function: ', time.time()-start_time)
    return relevant_tweet

In [99]:
## Get PI object using PI credentials - Aradhna
personality_insights = PersonalityInsightsV3(
version='2017-10-13',
username='04847f5d-afb4-4b08-8ed1-ae8ac7a56ac9',
password='8jniPKnFjDYO')

In [100]:
# Ram PI Credentials 
personality_insights = PersonalityInsightsV3(
version='2017-10-13',
username='f2264642-7379-488b-9154-e1373384cbc0',
password='rUBImsJrWStQ')

In [101]:
## Call Personality Insights on the tweets for the user
def getPersonality(tweets):
    # get tweets by user
    start_time = time.time()
    if tweets == None:
        profile = None
    else:
        tweets_content = ' '.join(tweets)
        # UTF-8 encoding
        twt = tweets_content.encode('utf-8')
        
        # call PI to get personality profile
        try:
            profile = personality_insights.profile(twt, content_type = 'text/plain', content_language = None,
                                           accept ='application/json', accept_language = None, raw_scores = False,
                                           consumption_preferences = False, csv_headers = False)
            
        except Exception:
            profile = None
    #print('Execution time for getPersonality function: ', time.time()-start_time)
    return profile

## Extract OCEAN percentiles from PI data
def extractOCEANtraits(profile):
    start_time = time.time()
    if profile == None:
        openness = None
        conscientiousness = None
        extraversion = None
        agreeableness = None
        neuroticism = None
    else:
        personality = profile['personality']
        openness = personality[0]['percentile']
        conscientiousness = personality[1]['percentile']
        extraversion = personality[2]['percentile']
        agreeableness = personality[3]['percentile']
        neuroticism = personality[4]['percentile']
        
    #print('Execution time for extractOCEANtraits function: ', time.time()-start_time)
    return openness, conscientiousness, extraversion, agreeableness, neuroticism

## Combine function calls for a user
def getPItraits(user):
    
    pitraits = {}
    try:
        ### Uncomment the line below to use FASTER language detection.
        
        #tweets = LanguageIdentifier_relevant_tweets(user)
        
        ### The line below gives a more ACCURATE language detection.
        tweets = detect_langs_relevant_tweets(user)
        # run PI profile on extracted tweets
        profile = getPersonality(tweets)
        # extract OCEAN traits
        traits = extractOCEANtraits(profile)
        if traits:
            # writing the personality profile of a user into dictionary
            OCEANTraits = dict(zip(['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism'],traits))
            return dict([(user,OCEANTraits)])
        else:
            return None
    except Exception:
        return None

#### looping around the celebrities list and converting the Personality traits result dictionary into a data frame


In [105]:
pitraits_dict = {}
### Restricted the celeb list to 4 to display final output. 
for celeb in celeb_list[:5]:
    PItraits = getPItraits(celeb)
    if PItraits is not None:
        #print(celeb)
        pitraits_dict.update(getPItraits(celeb))
    else:
        pass
traits_df = pd.DataFrame(pitraits_dict)
traits_df = traits_df.transpose()
traits_df


,agreeableness,conscientiousness,extraversion,neuroticism,openness
SrBachchan,0.741211,0.910018,0.291282,0.755362,0.818320
aamir_khan,0.958284,0.821784,0.580477,0.443622,0.535877
akshaykumar,0.998325,0.981034,0.933315,0.896125,0.611194
iamsrk,0.983320,0.716730,0.504650,0.549974,0.331970
